In [1]:
# Playing the bag file and extracting mediapipe landmark positions
import pyrealsense2 as rs
import numpy as np
import cv2
import os.path
import mediapipe as mp
import time
import os
from support.funcs import *

try:
    os.remove('posout.npy')
except:
    pass

# Setting the parameters of the stream
h=720 
w=1280
fps=30
windowscale=0.6

# Initializing the landmark lists
LS, LE, LW, RS, RE, RW, TR = [], [], [], [], [], [], []
RI,LI=[],[]

# Initializing the model to locate the landmarks
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
 
# Initializing the drawing utils for drawing the landmarks on image
mp_drawing = mp.solutions.drawing_utils

# Creating pointcloud object
pc = rs.pointcloud()


try:
    # Create pipeline
    pipeline = rs.pipeline()

    # Create a config object
    config = rs.config()

    # Tell config that we will use a recorded device from file to be used by the pipeline through playback.
    rs.config.enable_device_from_file(config, r'C:\Users\arpan\OneDrive\Documents\internship\bags\arpan_t2.bag', repeat_playback=False)

    # Configure the pipeline to stream the depth stream
    # Change this parameters according to the recorded bag file resolution
    config.enable_stream(rs.stream.depth, rs.format.z16, fps)
    config.enable_stream(rs.stream.color, rs.format.rgb8, fps)

    # Start streaming from file
    profile=pipeline.start(config)
    
    # Create colorizer object
    colorizer = rs.colorizer()

    #Needed so frames don't get dropped during processing:
    profile.get_device().as_playback().set_real_time(False)

    align_to = rs.stream.color
    align = rs.align(align_to)

    # Initializing the list of timestamps
    timestamps=[]

    # Number of frames 
    c=0

    # Streaming loop
    while True:

        frame_present, frameset = pipeline.try_wait_for_frames()
    
        #End loop once video finishes
        if not frame_present:
            break

        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue
        
        # get timestamp of frame
        timestamps.append((rs.frame.get_frame_metadata(depth_frame,rs.frame_metadata_value.time_of_arrival))/1000)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame() 
        color_frame = aligned_frames.get_color_frame()

        # Convert images to numpy arrays
        depth_image = np.asanyarray(aligned_depth_frame.get_data()) 
        color_image = np.asanyarray(color_frame.get_data()) 

        # Making predictions using holistic model
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        color_image.flags.writeable = False
        results = holistic_model.process(color_image)
        color_image.flags.writeable = True

        #converting color image to BGR
        color_image = cv2.cvtColor(color_image, cv2.COLOR_RGB2BGR)

        # Drawing the pose landmarks
        mp_drawing.draw_landmarks(
        color_image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS)

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        # Finding and saving the landmark positions        
        try:
            dic = {}
            for mark, data_point in zip(mp_holistic.PoseLandmark, results.pose_landmarks.landmark):
                dic[mark.value] = dict(landmark = mark.name, 
                    x = data_point.x,
                    y = data_point.y)        
            try:
                LS.append([dic[11]['x']*w,dic[11]['y']*h])
            except:
                LS.append(np.nan)
            try:
                LE.append([dic[13]['x']*w,dic[13]['y']*h])
            except:
                LE.append(np.nan)
            try:
                LW.append([dic[15]['x']*w,dic[15]['y']*h])
            except:
                LW.append(np.nan)
            try:
                RS.append([dic[12]['x']*w,dic[12]['y']*h])
            except:
                RS.append(np.nan)
            try:
                RE.append([dic[14]['x']*w,dic[14]['y']*h])
            except:
                RE.append(np.nan)
            try:
                RW.append([dic[16]['x']*w,dic[16]['y']*h])
            except:
                RW.append(np.nan)
            
            try:
                Smid=midpoint([dic[11]['x']*w,dic[11]['y']*h],[dic[12]['x']*w,dic[12]['y']*h])
                perpx=int(Smid[0])
                perpy=(int(Smid[1])+25)

                cv2.circle(color_image,(perpx,perpy) , 5, (0, 0, 255), 2)
                TR.append([perpx,perpy])     #in uv format  
            except:
                TR.append(np.nan)

            try:
                RI.append([dic[20]['x']*w,dic[20]['y']*h])
                LI.append([dic[19]['x']*w,dic[19]['y']*h])

                # Drawing the boxes around limbs for occlusion
                cv2.putText(color_image, str(int(c)), (50, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                draw_box(color_image,LS[c],LE[c])
                draw_box(color_image,RS[c],RE[c])
                draw_box(color_image,LE[c],LW[c])
                draw_box(color_image,RE[c],RW[c])
                draw_box(color_image,RW[c],([dic[20]['x']*w,dic[20]['y']*h]),(255,0,255),40)
                draw_box(color_image,LW[c],([dic[19]['x']*w,dic[19]['y']*h]),(0,255,255),40) 
            except:
                RI.append(np.nan)
                LI.append(np.nan)
        except:
            LS.append(np.nan)
            LE.append(np.nan)
            LW.append(np.nan)

            RS.append(np.nan)
            RE.append(np.nan)
            RW.append(np.nan)

            TR.append(np.nan) 
            RI.append(np.nan)
            LI.append(np.nan)
            pass
        
        # Finding the dimensions of the depth and colour image
        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)

        # resizing for display
        color_image=cv2.resize(color_image, (int(w*windowscale),int(h*windowscale)))
        depth_colormap=cv2.resize(depth_colormap, (int(w*windowscale),int(h*windowscale)))
        images =np.hstack((color_image, depth_colormap))
        
        # Show images
        cv2.putText(color_image, str(int(c)), (50, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

        # Calculating the point cloud
        mapped_frame = color_frame
        pc.map_to(mapped_frame)  
        try:
            points = pc.calculate(aligned_depth_frame)
            v = points.get_vertices()
            verts = np.asanyarray(v).view(np.float32)
            xyzpos=verts.reshape(h,w, 3)  # xyz
            xyzpos=xyzpos.astype(np.float16)
            # Saving the 3D position information in a file
            with open('posout.npy', 'ab') as f:
                np.save(f,xyzpos)   
                # print(c)         
        except:
            print(type(v))
        
        # Frame counter plus 1
        c+=1

finally:

    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()
    print(c)

496


In [2]:
# Saving landmark positions to a dataframe
import pandas as pd
import numpy as np

# Dictionary containing landmark names and corresponding values
land_marks = {'LS': LS, 'LE': LE, 'LW': LW, 'RS': RS, 'RE': RE, 'RW': RW, 'TR': TR}

pos = []
with open('posout.npy', 'rb') as f:
    for i in range(c):
        y = np.load(f)
        pos.append(y)

pos = np.array(pos)
print(pos.shape)

df = pd.DataFrame()
xyz = ['_x', '_y', '_z']

# Adding the 'epoch_time' column to the DataFrame using timestamps
df['epoch_time'] = pd.Series(timestamps)

# Iterate through each landmark in the dictionary
for key, value in land_marks.items():    
    for j in range(3):
        data = []
        # Iterate through each position in 'pos' array
        for i in range(len(pos)):
            try:
                # Extract the corresponding coordinate value from 'pos' array
                x = pos[i][int((value[i][1]))][int((value[i][0]))][j]
                data.append(x)
            except:
                continue
        # Add the extracted coordinate data as a new column in the DataFrame
        df[key + xyz[j]] = pd.Series(data)

df


(496, 720, 1280, 3)


,epoch_time,LS_x,LS_y,LS_z,LE_x,LE_y,LE_z,LW_x,LW_y,LW_z,...,RS_z,RE_x,RE_y,RE_z,RW_x,RW_y,RW_z,TR_x,TR_y,TR_z
0,1.685773e+09,-0.079773,-0.289551,1.438477,-0.047913,0.006790,1.481445,-0.021332,0.298584,1.448242,...,1.380859,-0.436768,0.038330,1.440430,-0.437744,0.314453,1.398438,-0.235962,-0.236694,1.378906
1,1.685773e+09,-0.078430,-0.288818,1.442383,-0.048279,0.011772,1.492188,-0.021210,0.296875,1.440430,...,1.378906,-0.435303,0.038055,1.429688,-0.431641,0.309814,1.377930,-0.235962,-0.236694,1.378906
2,1.685773e+09,-0.078308,-0.288330,1.440430,-0.047913,0.013313,1.481445,-0.021210,0.295166,1.440430,...,1.384766,-0.435303,0.039642,1.429688,-0.430908,0.309326,1.375977,-0.235107,-0.234375,1.374023
3,1.685773e+09,-0.078064,-0.287598,1.435547,-0.048065,0.014999,1.486328,-0.021332,0.296875,1.448242,...,1.380859,-0.435791,0.039673,1.431641,-0.431641,0.309570,1.382812,-0.235474,-0.236206,1.375977
4,1.685773e+09,-0.076843,-0.288818,1.442383,-0.048279,0.018341,1.492188,-0.022827,0.294189,1.442383,...,1.375977,-0.435547,0.044220,1.425781,-0.428711,0.307373,1.374023,-0.235474,-0.234619,1.375977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,1.685773e+09,-0.173340,-0.290771,1.375977,-0.126343,0.017563,1.427734,-0.083435,0.287109,1.392578,...,1.283203,-0.533203,0.058624,1.359375,-0.500488,0.326172,1.327148,-0.325684,-0.223633,1.278320
492,1.685773e+09,-0.172852,-0.289795,1.372070,-0.130127,0.017624,1.433594,-0.083191,0.286377,1.388672,...,1.276367,-0.525879,0.057617,1.335938,-0.499512,0.323730,1.317383,-0.327881,-0.224121,1.281250
493,1.685773e+09,-0.171509,-0.290283,1.374023,-0.129761,0.020737,1.429688,-0.085144,0.287842,1.396484,...,1.278320,-0.533691,0.058502,1.356445,-0.500488,0.322510,1.311523,-0.326416,-0.224121,1.281250
494,1.685773e+09,-0.170532,-0.291260,1.377930,-0.131470,0.020767,1.431641,-0.085266,0.288086,1.398438,...,1.275391,-0.532715,0.058411,1.353516,-0.499268,0.321777,1.308594,-0.327637,-0.224976,1.286133


In [3]:
# Finding and correcting occlusions based on boxes
startflag = 0  # Flag to track the starting frame of occlusion
before_occ = 0  # Frame before occlusion
do = False  # Flag indicating whether occlusion is occurring or not

box_dic = {
    'lub': [LS, LE, ['LS', 'LE']],
    'rub': [RS, RE, ['RS', 'RE']],
    'llb': [LE, LW, ['LE', 'LW']],
    'rlb': [RE, RW, ['RE', 'RW']],
    'lhb': [LW, LI, ['LW']],
    'rhb': [RW, RI, ['RW']]
}

# Iterate through each landmark and associated box in the dictionary
for k, j in land_marks.items():
    for key, values in box_dic.items():
        # Iterate through each frame
        for i in range(c):
            r = 40 if key == 'lhb' or key == 'rhb' else 30  # Radius for drawing the box

            try:
                # Check if the landmark is inside the box and not already occluded
                if point_in_quad(j[i], draw_box(color_image, values[0][i], values[1][i], (0, 0, 1), r)) and k not in values[2]:
                    if startflag == 0:
                        startflag = i
                        before_occ = startflag - 1  # Frame before occlusion

                    for p in values[2]:
                        if df[k+'_z'].tolist()[before_occ] > df[p+'_z'].tolist()[before_occ]:
                            print(k, 'is occluded by', key, p, 'at frame', i)
                            # Uncomment the following lines to correct the occlusion
                            # df.loc[i, k+'_x'] = df.loc[before_occ, k+'_y']
                            # df.loc[i, k+'_y'] = df.loc[before_occ, k+'_x']
                            df.loc[i, k+'_z'] = df.loc[before_occ, k+'_z']

                    do = True  # Set occlusion flag to True
            except:
                pass
            else:
                do = False  # Set occlusion flag to False

        if do:
            startflag = 0  # Reset the start flag to 0 for the next occlusion check


RE is occluded by rhb RW at frame 289
RE is occluded by rhb RW at frame 290
RE is occluded by rhb RW at frame 291
RE is occluded by rhb RW at frame 292
RE is occluded by rhb RW at frame 293
RE is occluded by rhb RW at frame 294
RE is occluded by rhb RW at frame 295
RE is occluded by rhb RW at frame 296
RE is occluded by rhb RW at frame 297
RE is occluded by rhb RW at frame 298
RE is occluded by rhb RW at frame 299
RE is occluded by rhb RW at frame 300


In [4]:
# Saving the dataframe as a csv
df.rename(columns={'LS_x':'ls_x','LE_x':'le_x','LW_x':'lw_x','RS_x':'rs_x','RE_x':'re_x','RW_x':'rw_x','TR_x':'tr_x',
                   'LS_y':'ls_y','LE_y':'le_y','LW_y':'lw_y','RS_y':'rs_y','RE_y':'re_y','RW_y':'rw_y','TR_y':'tr_y',
                   'LS_z':'ls_z','LE_z':'le_z','LW_z':'lw_z','RS_z':'rs_z','RE_z':'re_z','RW_z':'rw_z','TR_z':'tr_z'},inplace=True)
df.to_csv('mpipe.csv', index=False)
print(df.columns)


Index(['epoch_time', 'ls_x', 'ls_y', 'ls_z', 'le_x', 'le_y', 'le_z', 'lw_x',
       'lw_y', 'lw_z', 'rs_x', 'rs_y', 'rs_z', 're_x', 're_y', 're_z', 'rw_x',
       'rw_y', 'rw_z', 'tr_x', 'tr_y', 'tr_z'],
      dtype='object')
